# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
This project was done as an assignment for the Udacity course "Self-Driving Car Engineer Nanodegree". The purpose of the code in this project is to identify the lane lines in images/videos taken from inside a moving car and transform the image/video such that the lane lines are clearly marked.

<figure>
 <img src="test_images/solidYellowCurve.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Example of an input image </p> 
 </figcaption>
</figure>

<figure>
 <img src="test_images_output/solidYellowCurve-lines.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Output of the code </p> 
 </figcaption>
</figure>

## Import Packages
OpenCV (cv2) is an open-source library for computer vision.

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Pipeline for identification of lane lines

The pipeline for the identification of the lane lines consists of the following steps:
1. Since lane lines are either white or yellow, we apply a mask to the image that will single out the white and yellow lane lines.
2. We transform the image to the gray scale.
3. We apply a Gaussian filter in order to get rid of noise on the image.
4. We transform the image using Canny Edge detection. By taking the gradient of the pixels in the image this algorithm allows to detect edges in the image.
5. We mask all the pixels in the image outside our region of interest, a trapezoid on the bottom half of the image, where the lane lines must be.
6. Using the Hough transform we detect the lines in the image.
7. In order to mark the lines on the image/video, we do the following:
  1. We separate the line segments detected by the Hough algorithm into those that belong to the left lane line and those that belong to the right lane line.
  2. We fit a polynomial y = mx + b to the points belonging to the left line segments and another polynomial to the right line segments.
  3. For videos, in order to avoid flickering, we average the slope m and the y-intercept b over several frames.
  4. Using the polynomials we draw the left and right lines on the image.

## Helper Functions

The following functions are needed for the pipeline:

### Color masking

In [2]:
def mask_color(image, lower, upper):
    
    #Transform input image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    #Create the mask (pixel values must be in between the lower and upper boundaries)
    mask = cv2.inRange(hsv, lower, upper)
    #Apply mask to the input image 
    res = cv2.bitwise_and(image, image, mask = mask)
    
    return res

### Transform to grayscale

In [3]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

### Gaussian filter

In [4]:
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

### Canny edge detection

In [5]:
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

### Masking of pixels outside the region of interest

In [6]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count #declare tuple with value 255 and length channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    
    return masked_image

### Finding Hough lines and drawing them on the image

In [7]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, vertices, mLeft, bLeft, mRight, bRight, \
                avgFrames):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    #Using Hough transform detect the lines in the image
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, \
                            maxLineGap=max_line_gap)
    #Initialize the line image as an array of zeros
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    #Draw the lines on the input image
    draw_lines(line_img, lines, vertices, mLeft, bLeft, mRight, bRight, avgFrames)

    return line_img

In [8]:
def draw_lines(img, lines, vertices, mLeft, bLeft, mRight, bRight, avgFrames):
    """
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    """
    #separate the line segments detected with the Hough algorithm into left and right
    lls_x, lls_y, rls_x, rls_y = seperate_line_segments(lines)
    #fit a line to the left line segments
    fit_line(img, lls_x, lls_y, mLeft, bLeft, avgFrames, 0, vertices[0][1][0])
    #fit a line to the right line segments
    fit_line(img, rls_x, rls_y, mRight, bRight, avgFrames, vertices[0][2][0], img.shape[1])

In [9]:
def seperate_line_segments(lines):
    
    #Initialize arrays...
    #...x-values of left line segments
    left_line_segments_x = []
    #...y-values of left line segments
    left_line_segments_y = []
    #...x-values of right line segments
    right_line_segments_x = []
    #...y-values of right line segments
    right_line_segments_y = []
    
    #check if lines were found by the Hough algorithm
    if lines is not None:
        #loop over line segments in the output of the Hough algorithm
        for line in lines:
            #each line segment consists of two points that define the line, get their coordinates
            x1 = line[0][0]
            x2 = line[0][2]
            y1 = line[0][1]
            y2 = line[0][3]
            #Calculate the slope of the line segment
            slope = (y2-y1)/(x2-x1)
            #If the slope is negative the line segment belongs to the left line (the origin is in the upper left corner)
            if slope < 0 :
                left_line_segments_x.append(x1)
                left_line_segments_x.append(x2)
                left_line_segments_y.append(y1)
                left_line_segments_y.append(y2)
            #If the slope is positive, the line segment belongs to the right line
            else :
                right_line_segments_x.append(x1)
                right_line_segments_x.append(x2)
                right_line_segments_y.append(y1)
                right_line_segments_y.append(y2)
            
    return left_line_segments_x, left_line_segments_y, right_line_segments_x, right_line_segments_y

In [10]:
def fit_line(img, ls_x, ls_y, m, b, avgFrames, x0, x1, color=[255, 0, 0], thickness=8):    

    #check if the arrays of the x- and y-coordintes of the line segments are unempty
    if ((len(ls_x)>0) and (len(ls_y)>0)):
        #fit a polynomial y = mx + b to the points of the line segments
        line = np.polyfit(ls_x, ls_y, 1)
        #add the slope to the array of slopes
        m.append(line[0])
        #add the y-intercept to the array of y-intercepts
        b.append(line[1])
    
    #if the length of the m-array is greater the number of frames we want to average over, remove the first
    #element in the array
    if (len(m)>avgFrames) :
        m.pop(0)
    
    #if the m-array is not empty, average over it, else set m to zero
    if (m):
        m_av = np.average(m)
    else:
        m_av = 0
      
    #if the length of the b-array is greater the number of frames we want to average over, remove the first
    #element in the array    
    if (len(b)>avgFrames):
        b.pop(0)
    
    #if the b-array is not empty, average over it, else set b to zero
    if (b):
        b_av = np.average(b)
    else:
        b_av = 0
    
    #make a function with the average values of m and b
    f_line = np.poly1d([m_av, b_av])
     
    #for the x-values of the points at the beginning and the end of the line we want to draw, calculate the y-values
    #round and cast them to integer values
    p1 = (x0, int(round(f_line(x0))))
    p2 = (x1, int(round(f_line(x1))))
    
    #draw the lines on the image
    cv2.line(img, p1, p2, color, thickness)

In [11]:
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Assembling the pipeline

In [12]:
'''
This function takes an input image as well as the arrays of the slopes and y-intercepts of the left and right lines 
of the previous frames and returns an image with the lane lines detected 

'''
def line_detection(input_image, mLeft, bLeft, mRight, bRight):
    
    #lower and upper bounds for the color masking
    lower = np.array([0,0,230])
    upper = np.array([179,255,255])
    
    #color masking
    color_masked_image = mask_color(input_image, lower, upper)
    
    #transform image to gray scale
    gray_image = grayscale(color_masked_image)

    #to remove noise apply Gaussian smoothing to the image
    kernel_size = 15
    blurred_image = gaussian_blur(gray_image, kernel_size)

    #apply Canny edge detection
    low_threshold = 50
    high_threshold = 140
    canny_image = canny(blurred_image, low_threshold, high_threshold)

    #mask pixels outside region of interest
    
    #define vertices of region of interest
    ysize = canny_image.shape[0]
    xsize = canny_image.shape[1]
    
    vertices = np.array([[(int(round(0.1*xsize)), ysize),\
                        (int(round(0.45*xsize)), int(round(0.55*ysize))),\
                        (int(round(0.55*xsize)), int(round(0.55*ysize))),\
                        (int(round(0.9*xsize)), ysize)]],\
                        dtype=np.int32)
    
    #apply mask to the image
    masked_image = region_of_interest(canny_image, vertices)
    
    #Run Hough transform on Canny edge detected image

    #define parameters of Hough transform
    rho = 1 #this is the distance resolution of the accumulator in pixels
    theta = np.pi/180 #this is the angular resolution of the accumulator in pixels
    threshold = 7 #Accumulator threshold parameter. Only those lines are returned that get enough votes (>threshold)
    min_line_length = 25 #Minimum line length. Line segments shorter than that are rejected.
    max_line_gap = 1 #Maximum allowed gap in between points considered to be on the same line.
    avgFrames = 30 #number of frames to average lines over for videos
    
    line_image = hough_lines(masked_image, rho, theta, threshold, min_line_length, max_line_gap, vertices, \
                            mLeft, bLeft, mRight, bRight, avgFrames)

    #Overlay the image of the lines and the input image
    result_image = weighted_img(line_image, input_image)
    
    return result_image

## Test Images

Import images from the test_images folder, run the code on them and save them to the test_images_output folder.

In [13]:
import os
filenames_test_images = os.listdir("test_images/")


In [14]:
def save_line_image(input_file_name):
    
    #read image from file
    input_image = mpimg.imread(os.path.join("test_images", input_file_name))
    
    #initialize the arrays for the slopes and y-intercepts of the left and right lines
    mLeft = []
    bLeft = []
    mRight = []
    bRight = []
    
    #Detect lane lines and draw them on the image
    result_image = line_detection(input_image, mLeft, bLeft, mRight, bRight)
    
    #check if output folder exists, if not create it
    if not os.path.exists("test_images_output"):
        os.mkdir("test_images_output")
        
    #get filename of result image
    (head, tail) = os.path.split(input_file_name)
    (root, ext) = os.path.splitext(tail)
    result_filename = os.path.join("test_images_output", root + "-lines" + ext)
    
    #save the result image
    mpimg.imsave(result_filename, result_image)


In [15]:
for name in filenames_test_images:
    save_line_image(name)
    


## Test on Videos

In [29]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [30]:
def process_image(image):
    
    return line_detection(image, mLeft, bLeft, mRight, bRight)

Let's try the one with the solid white lane on the right first ...

In [31]:
#get video file name
white_output = 'test_videos_output/solidWhiteRight.mp4'

#initialize the arrays
mLeft = []
bLeft = []
mRight = []
bRight = []

#read in video
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")

#process the frames with the lane detection pipeline
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!

#write video to file
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:06<00:00, 32.62it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 4.24 s, sys: 404 ms, total: 4.64 s
Wall time: 7.89 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [32]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [33]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'

mLeft = []
bLeft = []
mRight = []
bRight = []

clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:20<00:00, 32.90it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 13 s, sys: 1.49 s, total: 14.5 s
Wall time: 21.5 s


In [34]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

The most challenging one...

In [35]:
challenge_output = 'test_videos_output/challenge.mp4'

mLeft = []
bLeft = []
mRight = []
bRight = []

clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:15<00:00, 15.92it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 8.18 s, sys: 1.12 s, total: 9.3 s
Wall time: 17.5 s


In [36]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))